In [40]:
# 1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv
import pandas as pd
data = pd.read_csv('data\\salary-train.csv')

In [41]:
# 2. Проведите предобработку

In [42]:
# Приведите тексты к нижнему регистру (text.lower())
data['FullDescription'] = data['FullDescription'].apply(lambda x: x.lower())
data['FullDescription'][:5]

0    international sales manager london ****k  ****...
1    an ideal opportunity for an individual that ha...
2    online content and brand manager// luxury reta...
3    a great local marketleader is seeking a perman...
4    registered nurse / rgn  nursing home for young...
Name: FullDescription, dtype: object

In [43]:
# Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова.
data['FullDescription'] = data['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data['FullDescription'][:5]

0    international sales manager london     k      ...
1    an ideal opportunity for an individual that ha...
2    online content and brand manager   luxury reta...
3    a great local marketleader is seeking a perman...
4    registered nurse   rgn  nursing home for young...
Name: FullDescription, dtype: object

In [52]:
# Примените TfidfVectorizer для преобразования текстов в векторы признаков. 
# Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=5)
X_train_text = vectorizer.fit_transform(data['FullDescription'])
words = vectorizer.get_feature_names()
words[1000:1010]

['airconditioned',
 'airconditioning',
 'aircraft',
 'airdrie',
 'airfare',
 'airfares',
 'airfield',
 'airframe',
 'airfreight',
 'airline']

In [51]:
# Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)
data[:5]

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


In [54]:
# Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_category = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_train_category[:5]

<5x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [55]:
# Объедините все полученные признаки в одну матрицу "объекты-признаки". 
# Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. 
# Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.
from scipy.sparse import hstack
X_train = hstack([X_train_text, X_train_category])

In [57]:
# 3. Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. 
# Целевая переменная записана в столбце SalaryNormalized.
from sklearn.linear_model import Ridge
y_train = data['SalaryNormalized']
model = Ridge(alpha=1, random_state=241)
model.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [62]:
# Постройте прогнозы для двух примеров из файла salary-test-mini.csv. 
# Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.
test = pd.read_csv('data\\salary-test-mini.csv')
test['FullDescription'] = test['FullDescription'].apply(lambda x: x.lower())
test['FullDescription'] = test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
X_test_text = vectorizer.transform(test['FullDescription'])
X_test_category = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([X_test_text, X_test_category])
y_test = model.predict(X_test)
print(round(y_test[0], 2), round(y_test[1], 2))
# 56555.62 37188.32

56555.62 37188.32
